In [1]:
%cd /homes/kb658/yinyang/

/homes/kb658/yinyang


In [2]:
import yaml
import jsonlines
import glob

config_path = "/homes/kb658/yinyang/configs/configs_os.yaml"

# Load config file
with open(config_path, 'r') as f:
    configs = yaml.safe_load(f)
    
artifact_folder = configs["raw_data"]["artifact_folder"]
mono_folder = configs["raw_data"]["mono_folder"]
json_folder = configs["raw_data"]["json_folder"]
raw_data_folders = configs["raw_data"]["raw_data_folders"]
data_path = raw_data_folders['classical']['folder_path']

# Get all the midi files in the data path recursively
midi_files = glob.glob(data_path + '/**/*.midi', recursive=True)

print(len(midi_files))

1276


In [180]:
from aria.data.midi import MidiDict
from aria.tokenizer import AbsTokenizer

tokenizer = AbsTokenizer()
midi_file_path = "/import/c4dm-datasets/maestro-v3.0.0/2008/MIDI-Unprocessed_07_R2_2008_01-05_ORIG_MID--AUDIO_07_R2_2008_wav--2.midi" # midi_files[1]
# midi_file_path = "/homes/kb658/yinyang/output/yin_yang/oestr019_YY.mid"
print(midi_file_path)
mid = MidiDict.from_midi(midi_file_path)
tokenized_sequence = tokenizer.tokenize(mid)

/import/c4dm-datasets/maestro-v3.0.0/2008/MIDI-Unprocessed_07_R2_2008_01-05_ORIG_MID--AUDIO_07_R2_2008_wav--2.midi


In [181]:
len(tokenized_sequence)

1549

In [182]:
tokenized_sequence[0:150]

[('prefix', 'instrument', 'piano'),
 '<S>',
 ('piano', 63, 75),
 ('onset', 0),
 ('dur', 920),
 ('piano', 64, 75),
 ('onset', 860),
 ('dur', 740),
 ('piano', 63, 75),
 ('onset', 1320),
 ('dur', 280),
 ('piano', 68, 60),
 ('onset', 1470),
 ('dur', 820),
 ('piano', 71, 75),
 ('onset', 1480),
 ('dur', 830),
 ('piano', 62, 75),
 ('onset', 1490),
 ('dur', 790),
 ('piano', 70, 75),
 ('onset', 2180),
 ('dur', 1090),
 ('piano', 61, 60),
 ('onset', 2180),
 ('dur', 1090),
 ('piano', 67, 60),
 ('onset', 2180),
 ('dur', 1080),
 ('piano', 39, 45),
 ('onset', 2180),
 ('dur', 1080),
 ('piano', 68, 60),
 ('onset', 3040),
 ('dur', 3280),
 ('piano', 44, 45),
 ('onset', 3060),
 ('dur', 3260),
 ('piano', 51, 30),
 ('onset', 4100),
 ('dur', 2220),
 ('piano', 56, 30),
 ('onset', 4120),
 ('dur', 2200),
 ('piano', 59, 30),
 ('onset', 4130),
 ('dur', 2200),
 ('piano', 63, 30),
 ('onset', 4130),
 ('dur', 2180),
 '<T>',
 ('piano', 70, 45),
 ('onset', 460),
 ('dur', 860),
 ('piano', 71, 60),
 ('onset', 840),
 ('du

In [183]:
# melody = tokenized_sequence[0:2] + tokenized_sequence[-11:-1] + tokenized_sequence[-1:]
# mid_dict = tokenizer.detokenize(melody) # mid_dict is a MidiDict object
# mid = mid_dict.to_midi() # mid is a mido.MidiFile
# mid.save('test_file.mid')

In [244]:
# Define a function to flatten the tokenized sequence
def flatten(sequence):
    flattened_sequence = []
    note_info = []
    for i in range(len(sequence)):
        if type(sequence[i]) == str:
            continue
            # flattened_sequence.append(sequence[i])
        if sequence[i][0] == "piano":
            note_info.append(sequence[i][1])
            note_info.append(sequence[i][2])
        elif sequence[i][0] == "onset":
            note_info.append(sequence[i][1])
        elif sequence[i][0] == "dur":
            note_info.append(sequence[i][1])
            flattened_sequence.append(note_info) 
            note_info = []

    return flattened_sequence

# Reverse the flattened function
def unflatten(sequence):
    unflattened_sequence = []
    for i in range(len(sequence)):
        note_info = ("piano", sequence[i][0], sequence[i][1])
        unflattened_sequence.append(note_info)
        note_info = ("onset", sequence[i][2])
        unflattened_sequence.append(note_info)
        note_info = ("dur", sequence[i][3])
        unflattened_sequence.append(note_info)
        note_info = []
        
        if i < len(sequence)-1:
            if ((sequence[i][2] + sequence[i][3]) >= 5000 and (sequence[i+1][2] + sequence[i+1][3]) < 5000) or (sequence[i+1][2] < sequence[i][2]):
                unflattened_sequence.append("<T>")

    return unflattened_sequence

In [245]:
tokenized_sequence[0:60]

[('prefix', 'instrument', 'piano'),
 '<S>',
 ('piano', 63, 75),
 ('onset', 0),
 ('dur', 920),
 ('piano', 64, 75),
 ('onset', 860),
 ('dur', 740),
 ('piano', 63, 75),
 ('onset', 1320),
 ('dur', 280),
 ('piano', 68, 60),
 ('onset', 1470),
 ('dur', 820),
 ('piano', 71, 75),
 ('onset', 1480),
 ('dur', 830),
 ('piano', 62, 75),
 ('onset', 1490),
 ('dur', 790),
 ('piano', 70, 75),
 ('onset', 2180),
 ('dur', 1090),
 ('piano', 61, 60),
 ('onset', 2180),
 ('dur', 1090),
 ('piano', 67, 60),
 ('onset', 2180),
 ('dur', 1080),
 ('piano', 39, 45),
 ('onset', 2180),
 ('dur', 1080),
 ('piano', 68, 60),
 ('onset', 3040),
 ('dur', 3280),
 ('piano', 44, 45),
 ('onset', 3060),
 ('dur', 3260),
 ('piano', 51, 30),
 ('onset', 4100),
 ('dur', 2220),
 ('piano', 56, 30),
 ('onset', 4120),
 ('dur', 2200),
 ('piano', 59, 30),
 ('onset', 4130),
 ('dur', 2200),
 ('piano', 63, 30),
 ('onset', 4130),
 ('dur', 2180),
 '<T>',
 ('piano', 70, 45),
 ('onset', 460),
 ('dur', 860),
 ('piano', 71, 60),
 ('onset', 840),
 ('du

In [246]:
# Call the flatten function
flattened_sequence = flatten(tokenized_sequence)
print(len(flattened_sequence))
print(flattened_sequence[-10:]) # [Midi pitch, Velocity, Onset time, Duration]

509
[[61, 60, 3600, 1200], [70, 60, 3610, 1200], [67, 45, 3610, 1190], [39, 45, 3620, 1190], [68, 60, 4590, 4340], [44, 45, 4600, 4330], [63, 45, 870, 3060], [59, 30, 890, 3040], [51, 30, 930, 3000], [56, 30, 940, 3000]]


In [247]:
from copy import deepcopy

overlap = []

for i in range(len(flattened_sequence)-1):
    onset_time = flattened_sequence[i][2]
    duration = flattened_sequence[i][3]
    offset_time = onset_time + duration
    next_onset_time = flattened_sequence[i+1][2]
    next_duration = flattened_sequence[i+1][3]
    # if offset_time >= 4990 and (next_onset_time + next_duration) < 5000:
    if next_onset_time + next_duration < offset_time:
        next_onset_time = 5000 + next_onset_time
    overlap_pct = (offset_time - next_onset_time) / duration
    overlap.append(overlap_pct)

# Retrograde the flattened sequence with respect to overlap
flattened_sequence_reversed = deepcopy(flattened_sequence)
flattened_sequence_reversed = flattened_sequence_reversed[::-1]
overlap_reversed = deepcopy(overlap)
overlap_reversed = overlap_reversed[::-1]
overlap_reversed = [0] + overlap_reversed

print(len(overlap_reversed))
print(overlap[-10:])
# If overlap > 0 then there is overlap. If overlap < 0 then there is a gap. If overlap = 0 then there is no overlap or gap.

509
[0.14285714285714285, 0.9916666666666667, -3.1666666666666665, 0.9915966386554622, 0.18487394957983194, 0.9976958525345622, 0.7066974595842956, 0.9934640522875817, 0.9868421052631579, 0.9966666666666667]


In [248]:
print(len(flattened_sequence_reversed))
print(len(overlap_reversed))

509
509


In [249]:
overlap_reversed[0:10]

[0,
 0.9966666666666667,
 0.9868421052631579,
 0.9934640522875817,
 0.7066974595842956,
 0.9976958525345622,
 0.18487394957983194,
 0.9915966386554622,
 -3.1666666666666665,
 0.9916666666666667]

In [250]:
# flattened_sequence

In [251]:
flattened_sequence_reversed[0:30]

[[56, 30, 940, 3000],
 [51, 30, 930, 3000],
 [59, 30, 890, 3040],
 [63, 45, 870, 3060],
 [44, 45, 4600, 4330],
 [68, 60, 4590, 4340],
 [39, 45, 3620, 1190],
 [67, 45, 3610, 1190],
 [70, 60, 3610, 1200],
 [61, 60, 3600, 1200],
 [62, 60, 2760, 980],
 [68, 60, 2760, 990],
 [71, 75, 2750, 1000],
 [63, 75, 2440, 480],
 [64, 75, 1880, 1010],
 [66, 60, 1790, 260],
 [64, 60, 1720, 330],
 [32, 30, 1080, 970],
 [44, 30, 1080, 980],
 [68, 45, 1060, 990],
 [63, 45, 1060, 1000],
 [71, 45, 1050, 1000],
 [49, 30, 310, 880],
 [64, 45, 310, 880],
 [37, 30, 310, 880],
 [68, 60, 300, 890],
 [73, 60, 280, 910],
 [70, 60, 100, 250],
 [71, 75, 4670, 570],
 [73, 75, 4570, 310]]

In [252]:
def quantize_to_nearest_tenth(value):
    return round(value / 10) * 10

def update_onset_times(flattened_seq, overlap):
    sequence = deepcopy(flattened_seq)
    
    for i in range(len(sequence)):
        if i == 0:
            sequence[i][2] = 0
        else:
            if overlap[i] > 0:
                # There is overlap
                previous_onset_end_time = sequence[i-1][2] + sequence[i-1][3]
                previous_duration = sequence[i-1][3]
                overlap_pct = overlap[i]
                updated_onset_time = previous_onset_end_time - (overlap_pct * previous_duration)
                if updated_onset_time >= 5000:
                    # Reset the onset time
                    updated_onset_time = updated_onset_time % 5000
                updated_onset_time = quantize_to_nearest_tenth(updated_onset_time)
                sequence[i][2] = updated_onset_time
            elif overlap[i] < 0: 
                # There is a gap
                previous_onset_time = flattened_seq[i-1][2]
                # previous_onset_end_time = flattened_seq[i-1][2] + flattened_seq[i-1][3]
                current_onset_time = sequence[i][2]
                current_duration = sequence[i][3]
                if current_onset_time > previous_onset_time:
                     previous_onset_time += 5000
                gap = previous_onset_time - (current_onset_time + current_duration)
                updated_previous_onset_end_time = sequence[i-1][2] + sequence[i-1][3]
                updated_onset_time = updated_previous_onset_end_time + gap
                if updated_onset_time >= 5000:
                    # Reset the onset time
                    updated_onset_time = updated_onset_time % 5000
                updated_onset_time = quantize_to_nearest_tenth(updated_onset_time)
                sequence[i][2] = updated_onset_time
            else:
                # There is no overlap
                previous_onset_end_time = sequence[i-1][2] + sequence[i-1][3]
                if previous_onset_end_time >= 5000:
                    # Reset the onset time
                    previous_onset_end_time = previous_onset_end_time % 5000
                sequence[i][2] = previous_onset_end_time

    return sequence

updated_sequence_reversed = update_onset_times(flattened_sequence_reversed, overlap_reversed)

In [253]:
# len(overlap_reversed)
len(flattened_sequence_reversed)

509

In [259]:
updated_sequence_reversed[0:50]

[[56, 30, 0, 3000],
 [51, 30, 10, 3000],
 [59, 30, 50, 3040],
 [63, 45, 70, 3060],
 [44, 45, 970, 4330],
 [68, 60, 980, 4340],
 [39, 45, 4520, 1190],
 [67, 45, 4530, 1190],
 [70, 60, 4520, 1200],
 [61, 60, 4530, 1200],
 [62, 60, 560, 980],
 [68, 60, 550, 990],
 [71, 75, 560, 1000],
 [63, 75, 1210, 480],
 [64, 75, 1480, 1010],
 [66, 60, 1830, 260],
 [64, 60, 1890, 330],
 [32, 30, 2110, 970],
 [44, 30, 2100, 980],
 [68, 45, 2120, 990],
 [63, 45, 2110, 1000],
 [71, 45, 2120, 1000],
 [49, 30, 2960, 880],
 [64, 45, 2960, 880],
 [37, 30, 2960, 880],
 [68, 60, 2970, 890],
 [73, 60, 2990, 910],
 [70, 60, 3650, 250],
 [71, 75, 3840, 570],
 [73, 75, 4020, 310],
 [44, 45, 4210, 770],
 [56, 45, 4230, 790],
 [68, 45, 4240, 790],
 [71, 45, 4250, 800],
 [75, 60, 4260, 810],
 [68, 30, 4970, 740],
 [49, 45, 4980, 750],
 [61, 45, 4990, 760],
 [70, 60, 5000, 770],
 [76, 75, 10, 780],
 [73, 75, 480, 250],
 [75, 75, 640, 710],
 [56, 30, 950, 1230],
 [51, 30, 960, 1240],
 [59, 30, 960, 1240],
 [63, 45, 970,

In [255]:
flattened_sequence[-50:-36]

[[63, 30, 1530, 1940],
 [56, 30, 1540, 1930],
 [59, 30, 1560, 1910],
 [71, 60, 2190, 1270],
 [63, 45, 2210, 1250],
 [59, 30, 2220, 1240],
 [51, 30, 2220, 1240],
 [56, 30, 2230, 1230],
 [75, 75, 2760, 710],
 [73, 75, 3220, 250],
 [76, 75, 3370, 780],
 [70, 60, 3380, 770],
 [61, 45, 3390, 760],
 [49, 45, 3400, 750]]

In [256]:
print(updated_sequence_reversed[0:3])
print(flattened_sequence[-3:])

[[56, 30, 0, 3000], [51, 30, 10, 3000], [59, 30, 50, 3040]]
[[59, 30, 890, 3040], [51, 30, 930, 3000], [56, 30, 940, 3000]]


In [257]:
retrograde = unflatten(updated_sequence_reversed)

melody = tokenized_sequence[0:2] + retrograde + tokenized_sequence[-1:]
mid_dict = tokenizer.detokenize(melody) # mid_dict is a MidiDict object
mid = mid_dict.to_midi() # mid is a mido.MidiFile
mid.save('test_file.mid')

In [258]:
retrograde

[('piano', 56, 30),
 ('onset', 0),
 ('dur', 3000),
 ('piano', 51, 30),
 ('onset', 10),
 ('dur', 3000),
 ('piano', 59, 30),
 ('onset', 50),
 ('dur', 3040),
 ('piano', 63, 45),
 ('onset', 70),
 ('dur', 3060),
 ('piano', 44, 45),
 ('onset', 970),
 ('dur', 4330),
 ('piano', 68, 60),
 ('onset', 980),
 ('dur', 4340),
 ('piano', 39, 45),
 ('onset', 4520),
 ('dur', 1190),
 ('piano', 67, 45),
 ('onset', 4530),
 ('dur', 1190),
 '<T>',
 ('piano', 70, 60),
 ('onset', 4520),
 ('dur', 1200),
 ('piano', 61, 60),
 ('onset', 4530),
 ('dur', 1200),
 '<T>',
 ('piano', 62, 60),
 ('onset', 560),
 ('dur', 980),
 '<T>',
 ('piano', 68, 60),
 ('onset', 550),
 ('dur', 990),
 ('piano', 71, 75),
 ('onset', 560),
 ('dur', 1000),
 ('piano', 63, 75),
 ('onset', 1210),
 ('dur', 480),
 ('piano', 64, 75),
 ('onset', 1480),
 ('dur', 1010),
 ('piano', 66, 60),
 ('onset', 1830),
 ('dur', 260),
 ('piano', 64, 60),
 ('onset', 1890),
 ('dur', 330),
 ('piano', 32, 30),
 ('onset', 2110),
 ('dur', 970),
 '<T>',
 ('piano', 44, 3

## Basic retrograde

In [45]:
# Reverse the elements in flattened_sequence and calculate onset times as 5000 - current onset
flattened_sequence_reversed = []
for i in range(len(flattened_sequence)):
    new_onset = 5000 - flattened_sequence[i][2] if flattened_sequence[i][2] != 0 else 4990
    flattened_sequence_reversed.append([flattened_sequence[i][0], flattened_sequence[i][1], new_onset, flattened_sequence[i][3]])
flattened_sequence_reversed = flattened_sequence_reversed[::-1]

print(flattened_sequence[-10:], '\n')
print(flattened_sequence_reversed[0:10])

[[61, 60, 3600, 1200], [70, 60, 3610, 1200], [67, 45, 3610, 1190], [39, 45, 3620, 1190], [68, 60, 4590, 4340], [44, 45, 4600, 4330], [63, 45, 870, 3060], [59, 30, 890, 3040], [51, 30, 930, 3000], [56, 30, 940, 3000]] 

[[56, 30, 4060, 3000], [51, 30, 4070, 3000], [59, 30, 4110, 3040], [63, 45, 4130, 3060], [44, 45, 400, 4330], [68, 60, 410, 4340], [39, 45, 1380, 1190], [67, 45, 1390, 1190], [70, 60, 1390, 1200], [61, 60, 1400, 1200]]


In [48]:
retrograde = unflatten(flattened_sequence_reversed)
print(retrograde)
print(tokenized_sequence)

[('piano', 56, 30), ('onset', 4060), ('dur', 3000), ('piano', 51, 30), ('onset', 4070), ('dur', 3000), ('piano', 59, 30), ('onset', 4110), ('dur', 3040), ('piano', 63, 45), ('onset', 4130), ('dur', 3060), '<T>', ('piano', 44, 45), ('onset', 400), ('dur', 4330), ('piano', 68, 60), ('onset', 410), ('dur', 4340), ('piano', 39, 45), ('onset', 1380), ('dur', 1190), ('piano', 67, 45), ('onset', 1390), ('dur', 1190), ('piano', 70, 60), ('onset', 1390), ('dur', 1200), ('piano', 61, 60), ('onset', 1400), ('dur', 1200), ('piano', 62, 60), ('onset', 2240), ('dur', 980), ('piano', 68, 60), ('onset', 2240), ('dur', 990), ('piano', 71, 75), ('onset', 2250), ('dur', 1000), ('piano', 63, 75), ('onset', 2560), ('dur', 480), ('piano', 64, 75), ('onset', 3120), ('dur', 1010), ('piano', 66, 60), ('onset', 3210), ('dur', 260), ('piano', 64, 60), ('onset', 3280), ('dur', 330), ('piano', 32, 30), ('onset', 3920), ('dur', 970), ('piano', 44, 30), ('onset', 3920), ('dur', 980), ('piano', 68, 45), ('onset', 394

In [47]:
melody = tokenized_sequence[0:2] + retrograde + tokenized_sequence[-1:]
mid_dict = tokenizer.detokenize(melody) # mid_dict is a MidiDict object
mid = mid_dict.to_midi() # mid is a mido.MidiFile
mid.save('test_file.mid')

## Skyline Algorithm

In [15]:
# Define a function to flatten the tokenized sequence
def flatten(sequence):
    flattened_sequence = []
    note_info = []
    for i in range(len(sequence)):
        if sequence[i] == "<T>":
            flattened_sequence.append(sequence[i])
        if sequence[i][0] == "piano":
            note_info.append(sequence[i][1])
            note_info.append(sequence[i][2])
        elif sequence[i][0] == "onset":
            note_info.append(sequence[i][1])
        elif sequence[i][0] == "dur":
            note_info.append(sequence[i][1])
            flattened_sequence.append(note_info) 
            note_info = []

    return flattened_sequence

def skyline(sequence: list, diff_threshold=50):
    melody = []
    pitch = 0
    velocity = sequence[0][1]
    onset = sequence[0][2]
    duration = sequence[0][3]
    i = 0

    for i in range(len(sequence)):
        if type(sequence[i]) != str:
            current_pitch = sequence[i][0]
            current_velocity = sequence[i][1]
            current_onset = sequence[i][2]
            current_duration = sequence[i][3]

            # Get next note onset
            if i+2 < len(sequence):
                if type(sequence[i+1]) == str and type(sequence[i+2]) == str:
                    next_onset = 5000
                elif type(sequence[i+1]) == str and type(sequence[i+2]) != str:
                    next_onset = sequence[i+2][2]
                else:
                    next_onset = sequence[i+1][2]
            else:
                next_onset = sequence[-1][2]
            
            # Get difference between current and next onset
            diff_next_curr_onset = abs(next_onset - current_onset)

            # Get difference between current and previous onset
            diff_curr_prev_onset = abs(current_onset - onset)

            # Check if current_onset is different from next onset
            if diff_next_curr_onset > diff_threshold:
                # Check if current_onset is different from previous onset and if current_pitch is greater than pitch
                if diff_curr_prev_onset < diff_threshold and current_pitch > pitch:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                elif diff_curr_prev_onset >= diff_threshold:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                else:
                    continue
                # Append the last note
                melody.append(("piano", pitch, velocity))
                melody.append(("onset", onset))
                melody.append(("dur", duration))

            else:
                if current_pitch > pitch:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                else:
                    continue
        else:
            melody.append(sequence[i])
    return melody
                
# Call the flatten function
flattened_sequence = flatten(tokenized_sequence)
print(flattened_sequence[0:100])

# Call the skyline function
melody = skyline(flattened_sequence, diff_threshold=30)
melody = tokenized_sequence[0:2] + melody + tokenized_sequence[-1:]
print(melody[0:100], '\n')

[[63, 75, 0, 920], [64, 75, 860, 740], [63, 75, 1320, 280], [68, 60, 1470, 820], [71, 75, 1480, 830], [62, 75, 1490, 790], [70, 75, 2180, 1090], [61, 60, 2180, 1090], [67, 60, 2180, 1080], [39, 45, 2180, 1080], [68, 60, 3040, 3280], [44, 45, 3060, 3260], [51, 30, 4100, 2220], [56, 30, 4120, 2200], [59, 30, 4130, 2200], [63, 30, 4130, 2180], '<T>', [70, 45, 460, 860], [71, 60, 840, 480], [73, 75, 1110, 300], [75, 75, 1360, 620], [78, 75, 1840, 180], [76, 60, 1940, 780], [75, 60, 2570, 890], [80, 60, 2950, 510], [75, 60, 3470, 2070], [68, 45, 4300, 1240], [71, 45, 4330, 1210], [63, 30, 4340, 1200], [56, 30, 4340, 1200], [75, 75, 4920, 620], '<T>', [73, 75, 370, 180], [76, 75, 500, 720], [70, 60, 500, 140], [61, 45, 520, 700], [49, 60, 530, 700], [68, 45, 530, 690], [75, 75, 1130, 660], [68, 45, 1150, 640], [71, 45, 1160, 150], [56, 45, 1160, 630], [44, 45, 1170, 620], [73, 60, 1630, 160], [71, 60, 1740, 600], [70, 60, 2220, 220], [73, 60, 2390, 810], [68, 45, 2410, 790], [64, 45, 2410, 7

In [16]:
melody[-10:]

[('onset', 2440),
 ('dur', 480),
 ('piano', 39, 45),
 ('onset', 3620),
 ('dur', 1190),
 '<T>',
 ('piano', 59, 30),
 ('onset', 890),
 ('dur', 3040),
 '<E>']

In [19]:
print(tokenized_sequence[1:150])

['<S>', ('piano', 67, 90), ('onset', 0), ('dur', 570), ('piano', 65, 90), ('onset', 560), ('dur', 530), ('piano', 70, 90), ('onset', 1070), ('dur', 550), ('piano', 72, 90), ('onset', 1580), ('dur', 250), ('piano', 77, 90), ('onset', 1820), ('dur', 290), ('piano', 74, 90), ('onset', 2070), ('dur', 550), ('piano', 72, 90), ('onset', 2580), ('dur', 280), ('piano', 77, 90), ('onset', 2830), ('dur', 300), ('piano', 74, 90), ('onset', 3100), ('dur', 550), ('piano', 70, 90), ('onset', 3600), ('dur', 550), ('piano', 72, 90), ('onset', 4120), ('dur', 590), ('piano', 67, 90), ('onset', 4660), ('dur', 550), '<T>', ('piano', 65, 90), ('onset', 180), ('dur', 580), ('piano', 67, 90), ('onset', 730), ('dur', 500), ('piano', 55, 60), ('onset', 770), ('dur', 470), ('piano', 58, 60), ('onset', 770), ('dur', 470), ('piano', 43, 60), ('onset', 770), ('dur', 470), ('piano', 62, 45), ('onset', 770), ('dur', 460), ('piano', 65, 75), ('onset', 1240), ('dur', 530), ('piano', 45, 60), ('onset', 1260), ('dur', 5

In [17]:
print(len(tokenized_sequence))
print(len(melody))

1549
504


In [18]:
mid_dict = tokenizer.detokenize(melody) # mid_dict is a MidiDict object
mid = mid_dict.to_midi() # mid is a mido.MidiFile
mid.save('test_file.mid')

In [36]:
from musicpy import *

midi_file_path = "/import/c4dm-datasets/maestro-v3.0.0/2008/MIDI-Unprocessed_07_R2_2008_01-05_ORIG_MID--AUDIO_07_R2_2008_wav--2.midi"
a, bpm, start_time = read(midi_file_path).merge()

In [66]:
# example_melody_index = a.split_melody(mode='index')
example_melody_index, example_chords_index = a.split_all(mode='index')

In [71]:
a.notes

[D#4,
 E4,
 D#4,
 G#4,
 B4,
 D4,
 C#4,
 A#4,
 D#2,
 G4,
 G#4,
 G#2,
 D#3,
 G#3,
 B3,
 D#4,
 A#4,
 B4,
 C#5,
 D#5,
 F#5,
 E5,
 D#5,
 G#5,
 D#5,
 G#4,
 B4,
 G#3,
 D#4,
 D#5,
 C#5,
 E5,
 A#4,
 C#4,
 C#3,
 G#4,
 D#5,
 G#4,
 B4,
 G#3,
 G#2,
 C#5,
 B4,
 A#4,
 C#5,
 E4,
 G#4,
 C#3,
 C#2,
 B4,
 D#4,
 G#1,
 G#4,
 G#2,
 E4,
 D#4,
 B4,
 G#4,
 D4,
 A#4,
 C#4,
 G4,
 D#2,
 G#2,
 G#4,
 D#3,
 B3,
 D#4,
 G#3,
 B4,
 B3,
 D#3,
 D#4,
 G#3,
 D#5,
 C#5,
 E5,
 A#4,
 C#4,
 G#4,
 C#3,
 G#4,
 D#5,
 G#4,
 B4,
 G#3,
 G#2,
 C#5,
 B4,
 A#4,
 C#5,
 C#3,
 C#2,
 G#4,
 E4,
 B4,
 D#4,
 G#4,
 G#1,
 G#2,
 E4,
 F#4,
 E4,
 D#4,
 B4,
 G#4,
 D4,
 A#4,
 C#4,
 G4,
 D#2,
 G#2,
 G#4,
 G#3,
 D#4,
 B3,
 B4,
 E4,
 G3,
 A#4,
 F#3,
 E4,
 C#4,
 B3,
 G#4,
 A#3,
 F#4,
 B4,
 D#4,
 A#4,
 E4,
 C#4,
 F#3,
 B3,
 G#4,
 F#4,
 A#3,
 G#4,
 B3,
 A#4,
 F#3,
 C#4,
 E4,
 F#4,
 B4,
 D#4,
 C#5,
 E4,
 F#4,
 A#4,
 F#3,
 C5,
 D#4,
 A4,
 C#5,
 C#4,
 F#4,
 A#4,
 F#3,
 G#5,
 F4,
 D4,
 B4,
 F#5,
 C#4,
 A#4,
 D#4,
 C#4,
 B4,
 G#4,
 F5,
 B5,
 C#5,
 A#5,
 F#4,
 

In [72]:
# Use index on a to get the melody
example_melody_individual = copy(a)
example_melody_individual.notes = [example_melody_individual.notes[i] for i in example_melody_index]
example_melody_individual.interval = [example_melody_individual.interval[i] for i in example_melody_index]
print(example_melody_individual)

chord(notes=[D#4, E4, D#4, G#4, B4, A#4, G4, G#4, D#4, A#4, ...], interval=[219/512, 59/256, 19/256, 5/768, 7/1536, 1/256, 655/1536, 3/256, 169/256, 73/384, ...], start_time=0)


In [73]:
write(example_melody_individual, bpm, name='example_melody.mid')